In [1]:
#Quiz 2
%load_ext autoreload
%autoreload 2
%matplotlib inline
from my_modules import risk
from my_modules import data
from my_modules import portfolio

from scipy.optimize import minimize
import numpy as np
import pandas as pd

In [36]:
#Use the EDHEC Hedge Fund Indices data set that we used in the lab assignment as well as in the previous week’s 
# assignments. Load them into Python and perform the following analysis based on data since 2000 (including 
# all of 2000):
hfi = data.get_hfi_returns()['2000':]
er = risk.annualise_rets(hfi)
cov = hfi.cov()

In [38]:
# 1. What was the Monthly Parametric Gaussian VaR at the 1% level (as a +ve number)
# of the Distressed Securities strategy?
risk.var_gaussian(hfi, level=1)['Distressed Securities'] #0.031430

0.03143017693928047

In [39]:
# 2. What was the 1% VaR for the same strategy after applying the Cornish-Fisher Adjustment?
risk.var_gaussian(hfi, level=1, modified=True)['Distressed Securities']

0.04966909110627918

In [40]:
# 3. Monthly Historic VaR at the 1% level (as a +ve number) of the Distressed Securities strategy?
risk.var_historic(hfi, level=1)['Distressed Securities']

0.042642

>Next, load the 30 industry return data using the erk.get_ind_returns() function that we developed during the lab sessions. For purposes of the remaining questions, use data during the 5 year period 2013-2017 (both inclusive) to estimate the expected returns as well as the covariance matrix. To be able to respond to the questions, you will need to build the MSR, EW and GMV portfolios consisting of the “Books”, “Steel”, "Oil", and "Mines" industries. Assume the risk free rate over the 5 year period is 10%.

In [46]:
l=['Books', 'Steel', 'Oil', 'Mines']
risk_free = 0.1
ind = data.get_ind30_vw_returns()['2013':'2017']
er = risk.annualise_rets(ind)
cov = ind.cov()

In [79]:
# 4. What is the weight of Steel in the EW Portfolio?
print("0.25")

0.25


In [80]:
# 5. What is the weight of the largest component of the MSR portfolio?
portfolio.msrp(risk_free, er[l], cov.loc[l,l])
print("1")

1


In [49]:
# 6. Which of the 4 components has the largest weight in the MSR portfolio?
print("Steel")

Steel


In [58]:
# 7. How many of the components of the MSR portfolio have non-zero weights?
print("One - the others are negilable")

One - the others are negilable


In [59]:
# 8. What is the weight of the largest component of the GMV portfolio?
cov

,Books,Steel,Oil,Mines
Books,0.002720,0.001940,0.001637,0.001539
Steel,0.001940,0.004391,0.002122,0.003391
Oil,0.001637,0.002122,0.002697,0.002179
Mines,0.001539,0.003391,0.002179,0.005232


In [60]:
portfolio.gmv(cov)

array([0.47702487, 0.        , 0.43411203, 0.0888631 ])

In [62]:
# 9. Which of the 4 components has the largest weight in the GMV portfolio?
print("Books")

Books


In [63]:
# 10. How many of the components of the GMV portfolio have non-zero weights?
print("3")

3


>Assume two different investors invested in the GMV and MSR portfolios at the start of 2018 using the weights we just computed. Compute the annualized volatility of these two portfolios over the next 12 months of 2018? (Hint: Use the portfolio_vol code we developed in the lab and use ind[“2018”][l].cov() to compute the covariance matrix for 2018, assuming that the variable ind holds the industry returns and the variable l holds the list of industry portfolios you are willing to hold. Don’t forget to annualize the volatility)

In [68]:
ind_18 = data.get_ind30_vw_returns()["2018"][l]
ind_18

,Books,Steel,Oil,Mines
2018-01,0.0449,0.0361,0.0286,0.0357
2018-02,-0.0325,-0.0199,-0.1102,-0.0335
2018-03,0.0031,-0.0586,0.0308,-0.0015
2018-04,-0.0062,0.0366,0.1030,-0.0339
2018-05,-0.0061,0.0562,0.0343,0.0308
2018-06,0.0218,-0.0514,0.0052,-0.0178
2018-07,-0.0417,0.0424,0.0104,-0.0224
2018-08,-0.0447,-0.0323,-0.0295,-0.1075
2018-09,-0.0221,-0.0067,0.0356,-0.0173
2018-10,-0.0666,-0.1161,-0.1206,-0.1032


In [69]:
ind_18_cov = ind_18.cov()
ind_18_cov

,Books,Steel,Oil,Mines
Books,0.002799,0.002098,0.002558,0.001629
Steel,0.002098,0.004027,0.003473,0.001843
Oil,0.002558,0.003473,0.005296,0.001961
Mines,0.001629,0.001843,0.001961,0.002059


In [81]:
# 11. What would be the annualized volatility over 2018 using the weights of the MSR portfolio?

# Steel was the only asset that didn't have a non-0 weighting, however I want to test the multiplication
vol_steel = risk.annualise_vol(ind_18['Steel'])
vol_steel

0.21983988181484185

In [82]:
weights_msr = portfolio.msrp(risk_free, er[l], cov.loc[l,l])
returns = risk.annualise_rets(ind_18)
#portfolio.returns(weights_msr, returns)
#portfolio.volatility(weights_msr, ind_18_cov) <- Annualize = 0.06346230747220724
ans = portfolio.volatility(weights_msr, ind_18_cov) * (12 ** 0.5)
ans

0.21983988181484188

In [83]:
# 12. What would be the annualized volatility over 2018 using the weights of the GMV portfolio? 
weights_gmv = portfolio.gmv(ind_18_cov)
ans = portfolio.volatility(weights_gmv, ind_18_cov) * (12 ** 0.5)
ans

0.15269232284490572